QISHA_GenROIS
Created by JFranco, 14 AUG 2023

This Python notebook reads in a seed image and uses Cellpose to segment the image into ROIs. It generates a map of the ROIs and saves the relevent info for use in Image J. The code is based on code written by JFranco (Github repository CaTA). 

In [5]:
#                           *** LIBRARIES ***
import datetime
import numpy as np
import os
import matplotlib.pyplot as plt
#from cellpose import models , plot
#import imageio
import pandas as pd
#import glob
#from scipy.ndimage import gaussian_filter
#from scipy import stats
#from skimage import io

In [10]:
# Starting the code off very simply with doing the most basic steps of reading in one seeding, 
#  running Cellpose, and then saving the ROI list to the directory

#                   *** WHERE TO GET/SAVE DATA ***   
# Directories 
#   UPDATE THIS INFO TO REFLECT YOUR SYSTEM
dirMain = "/Users/joyfranco/Dropbox (Partners HealthCare)/JF_Shared/Data/CYS/CYS_003/"
dirSI = dirMain+'SeedImages/'

#   ones that need to be made
dirROIs = dirMain+'ROIs/'
dirMasks = dirMain+"Masks/"                                                     # For storing cellpose masks 

# Create directory for storing ROIs and Masks
    if not os.path.exists(dirROIs):
        os.mkdir(dirROIs)
        os.mkdir(dirMasks)

#                   *** BEGIN PROCESS ***   
# Go to the location of the seed images and get a list
os.chdir(dirSI)
ims = os.listdir()

for im in ims:
    # Get the filename base
    fnBase = ims[0].split(".SeedImage",1)[0]

    
print(fnBase)

# Export the masks as an ROI archive file for ImageJ
#io.save_rois(masks, '<your_filename_string>')

['CYS_003.S11.TS4.SeedImage.png', 'CYS_003.S10.TS5.SeedImage.png', 'CYS_003.S11.TS2.SeedImage.png']
CYS_003.S11.TS4


In [ ]:
# This block of code is just copy/pasted from CaTA to help with writing the code for generating masks from cellpose
"""
#                   *** GET TIME OF ANALYSIS START ***
# could turn this into a module?
toa = str(datetime.datetime.today()).split()                                   #Time of analysis
today = toa[0]
now = toa[1]
timestamp = today.replace('-','')+'-'+now.replace(':','')[:6]

#                   *** ANALYSIS PARAMETERS ***   
# Directories 
#   UPDATE THIS INFO TO REFLECT YOUR SYSTEM
dirMain = "/Users/joyfranco/Dropbox (Partners HealthCare)/JF_Shared/Data/CodeDev/QISHA/"
dirData = dirMain+"RawImageFiles/"
dirSI = dirMain+'SeedImages/'

#   ones that need to be made
dirMasks = dirPrep+"Masks/"                                                     # For storing cellpose masks 

# Go to where raw .nd2 files are stored
os.chdir(dirData)
for samp in samps:
    fnBase = samp
                                                                                
    # Files 
    fnSI = fnBase+'.SeedImage.png' 
    fnMskCSV = fnBase+'.Masks.csv'
    fnMskArea = fnBase+'.MaskAreas.csv'
    fnMskLab = fnBase+'.MasksLabeled.png'
    fnStimVals = fnBase+'.StimVals.csv'        

    # Create directories
    if not os.path.exists(dirMasks):
        os.mkdir(dirMasks)

    #***initialize dataframes***
    dfAreas = pd.DataFrame()
    
    #                *** GENERATE ROI MASK FROM CALIBRATION FILE **
    
    #       ***         GET LIST OF IMAGES TO ANALYZE       ***                                                

    # Create cell pose model
    #   model_type='cyto' or model_type='nuclei'
    model = models.Cellpose(gpu=False, model_type='cyto')
    chan=[0,0]
    #   read in the seed image 
    img = io.imread(dirSI+fnSI)
    #   generate the masks using the seed image as the input for the model 
    masks, flows, styles, diams = model.eval(img, diameter=100, channels=chan,
                                             invert=False)
    #   create a dataframe based on the masks and save to csv
    pd.DataFrame(masks).to_csv(dirMasks+fnMskCSV)                                  # I don't love that the index and col heads are being saved
    
    #               *** USE MASKS TO GET MEAN GREY ***
    # arrays for storing mean grey values for cal and stim files
    mnsStim = []

    # arrays for storing median xs ys of mask labels 
    labels = np.array([0,0,0])
    labels = labels.reshape(1,3)

    # iterate over unique rois in the mask
    for roi in np.unique(masks):
        # In the future we will get rid of any cells that are too small, but
        #   for now we just want to save the data on their areas
        
        if roi != 0:  # skip background
            
            # setup the roiID to extend off prep+sample
            roiID = fnBase+'.'+str(roi)
            
            # get y & x coords of the roi
            ys, xs = np.nonzero(masks == roi)
            
            # need to store the median ys and xs for labeling rois
            x = [float(roi),np.median(xs),np.median(ys)]        
            x = np.array(x)
            labels = np.concatenate((labels,x.reshape((1,3))),axis=0)     
            
            # index into last two dimensions of the original data to get the raw values
            #    returns an array of size (nrows=frames,ncol=pixels)
            valsStim = stimND[..., ys, xs] 
            
            # average over the last axis (which will be the size of the ROI, but flattened)
            # these have mean pixel val for the roi, for both channels, and every frame
            #   The result here is an array of size (nrows=frames,ncols=1) <- because there's only one channel
            #   The operation takes the mean of each row (so it's the mean of all pixels at that roi,  that frame)
            mnsStim.append(valsStim.mean(axis=-1))

            # PROCESS VALUES FOR STIMULATION FILE
            # Get mean values for each ROI/channel/timepoint into dataframe
            # Assumes that means are ordered as [440 ex, 480 ex]
            # frames = indices and cols = val type (e.g., ratio)
            
            # REMEBER - This process gets handled differently when you 
            #   start with two channels rather than having a single channel
            #   tiff
            dfTStim = pd.DataFrame(mnsStim[roi-1],columns=(
                roiID+'.C1',roiID+'.C2'))
            # calculate the ratio column
            dfTStim[roiID+'.R'] = dfTStim[roiID+'.C1'] / dfTStim[roiID+'.C2']
            # calculate the baseline value
            baseline = dfTStim[roiID+'.R'][blSt:blEn].mean()
            # calculate fold change from baseline
            dfTStim[roiID+'.FC'] = (dfTStim[roiID+'.R'] - baseline)/ baseline
            
            # Have to reshape the cols to rows 
            dfStimVals = dfStimVals.append(dfTStim.T)
            
            # GET ROI AREA AND ADD TO DF
            df = pd.DataFrame([[roiID,len(xs)]],columns=['roiID','area_sqpix'])
            dfAreas = dfAreas.append(df)
    
    #       *** WRITE ALL MEASURED VALUES TO FILE ***
    # Save the quantified value for each blob
    dfStimVals.to_csv(dirVals+fnStimVals)
    dfAreas.to_csv(dirMasks+fnMskArea)
            
    #       *** CREATE MASK IMAGE WITH LABELS ***        
    
    fig, axes = plt.subplots(1, 2, dpi=300)
    fig.suptitle('Cellpose Segmentation Results')
    #plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=.5, wspace=0.4)
    plt.style.use('dark_background')
    
    # Plot #1: the seed for cellpose
    ax2 = plt.subplot(121)       
    ax2.set_title('Cellpose Seed Image')
    ax2.set_xticks([])
    ax2.set_yticks([])
    #box = ax2.get_position()
    #box.y0 = box.y0 - 0.1
    #box.y1 = box.y1 - 0.1
    #ax2.set_position(box)
    ax2.set_xlabel('')
    ax2.set_ylabel('')          
    plt.imshow(img, interpolation='nearest')                              #subplot(nrows, ncols, index, **kwargs)
    
    # Labeled image
    ax1 = plt.subplot(122)
    ax1.set_title('Identified ROIs')
    ax1.set_xticks([])
    ax1.set_yticks([])
    #box = ax1.get_position()
    #box.y0 = box.y0 - 0.1
    #box.y1 = box.y1 - 0.1
    #ax1.set_position(box)
    ax1.set_xlabel('')
    ax1.set_ylabel('')
    plt.imshow(masks)
    for i in range(0,len(labels)):
        plt.text(labels[i][1], labels[i][2], str(round(labels[i][0])),
                 ha="center", va="center", size=8,
                 bbox=dict(boxstyle="round",
                           ec=(1., 0.5, 0.5),
                           fc='w',
                           #fc=(1., 0.8, 0.8),
                           alpha=0.7
                           ), 
                 color ='black')                                                  #subplot(nrows, ncols, index, **kwargs)
    
    
    plt.savefig(dirMasks+fnMskLab, dpi='figure', format="png", metadata=None,
            bbox_inches=None, pad_inches=0.1,
            facecolor='auto', edgecolor='auto',
            backend=None)

    # Save summary image of masks and show to user
    fig = plt.figure(figsize=(12,5))
    plot.show_segmentation(fig, img, masks, flows[0], channels=chan)
    plt.tight_layout()
    plt.savefig(dirMasks+fnCP, dpi='figure', format="png", metadata=None,
            bbox_inches=None, pad_inches=0.05,
            facecolor='auto', edgecolor='auto',
            backend=None)

    

plt.close('all')
"""
